In [9]:
from ortools.sat.python import cp_model
import itertools
import numpy as np
import pandas as pd

In [12]:
def init_employee(model, timeslots=4*11, days=5):
    """
    Set up one employee
    """
    # Create a list of all possible shifts
    emplyee_times = [[],]*days
    for i in range(days):
        for j in range(timeslots):
            emplyee_times[i].append(model.NewBoolVar('e%i_%i' % (i, j)))
    return emplyee_times

def create_no_gaps_constraint(model, emplyee_times):
    """
    Create a constraint that there are no gaps in the schedule
    """
    # First add a new variable that is 0 (false) if the employee works at both index i and i+1 and 1 if they are working at i and not at i+1

    DAYS = len(emplyee_times)
    TIMESLOTS = len(emplyee_times[0])

    # Create a list of all possible shifts
    emplyee_no_gaps_constraint = [[],]*DAYS
    for i in range(DAYS):
        for j in range(TIMESLOTS-1):
            emplyee_no_gaps_constraint[i].append(model.NewBoolVar('e%i_%i' % (i, j)))
            
            # Add a constraint that this variable is true if the employee is working at index i but not i+1
            model.Add(emplyee_times[i][j] > emplyee_times[i][j+1]).OnlyEnforceIf(emplyee_no_gaps_constraint[i][j])

    # Add a constraint that the sum of all these variables is at most 1
    for i in range(DAYS):
        model.Add(sum(emplyee_no_gaps_constraint[i]) <= 1)

def create_desired_times_constraint(model, employees_times, desired_times):
    """ 
    Create a constraint that at least desired_times[i,j] (integer) employees must work at timeslot j on day i 
    """
    N_EMPLOYEES = len(employees_times)
    DAYS = len(employees_times[0])
    TIMESLOTS = len(employees_times[0][0])

    for i in range(DAYS):
        for j in range(TIMESLOTS):
            model.Add(sum(employees_times[k][i][j] for k in range(N_EMPLOYEES)) >= desired_times[i][j])


In [ ]:
def init_schedule(DAY_START, DAY_END, dM):
    timeslots = {"{}:{}".format(h, m): i for i, (h, m) in enumerate(
    itertools.product(range(DAY_START, DAY_END), range(0, 60, dM)))}
    ind_to_timeslot = {i: t for t, i in timeslots.items()}
    schedule = pd.DataFrame(0, index=timeslots.keys(), columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
    return np.array(schedule), schedule, timeslots, ind_to_timeslot

In [6]:
# Define employees and shifts
employees = {"Alice": 0, "Bob": 1, "Carol": 2, "Dave": 3, "Ed": 4}

# Define the timeslots such that there is one slot per M minutes
M = 15
assert 60 % M == 0
DAY_START = 6
DAY_END = 18
timeslots = {"{}:{}".format(h, m): i for i, (h, m) in enumerate(
    itertools.product(range(DAY_START, DAY_END), range(0, 60, M)))}
ind_to_timeslot = {i: t for t, i in timeslots.items()}

In [10]:
# Create a pandas dataframe to store the schedule with the first column for each timeslot, and the other colmuns are zero columns for 5 days
schedule = pd.DataFrame(0, index=timeslots.keys(), columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])

In [11]:
A = np.array(schedule)